<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.877 · Análisis de sentimientos y textos</p>
<p style="margin: 0; text-align:right;">Máster Universitario de Ciencia de Datos(Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# PRA 3: Deep Learning para el análisis de textos 

En esta práctica revisaremos y aplicaremos los conocimientos aprendidos durante el curso y, en más detalle, en los últimos módulos del mismo. En concreto trataremos los siguientes temas:

1. **Traducción automatica**: con custom embeddings y con embeddings preentrenados.
2. **Classificación de frases**: Aplicación de los conceptos ya trabajados para la reutilización de la arquitectura de dos modelos.

3. **Detección de NER y NEL**: detección y clasificación de entidades nombradas (NER) y entity linking basandonos en los temas ya trabajados en los notebooks de NER y NEL y añadiendo un ejemplo sencillo de transformers. 

También incluimos algunos otros temas transversales trabajados a lo largo de la asignatura.


<div class="alert alert-block alert-info">
<strong>Nombre y apellidos: EDUARDO MORA GONZÁLEZ</strong>
</div>

# PARTE 1

En esta primera parte de la práctica se pide resolver los ejercicios usando la libreria **KERAS**.

# 1. Traducción Automática (4 puntos)




## 1.1 TA con Custom Embeddings (2 puntos)

In [2]:
from numpy import array
import numpy as np
import string
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector, Flatten, Dense, Dropout, Activation
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from keras.models import load_model
import pandas as pd


El objetivo de este apartado es entrenar un modelo de traducción automática entre inglés y holandes, siguento los mismos pasos que en el notebook de Machine Translation. 


<strong>Implementación:</strong> Siguiendo los pasos trabajados en el notebook de traducción automática, implementar y entrenar un modelo de traducción automática, del inglés al holandés.  <br>
    - La capa embedding debe de tener una dimensión igual a 300 <br>
    - Se recomienda una longitud màxima de secuencia de 12 <br>
 <br>
    
Mostrad la aplicación del modelo entrenado con algún ejemplo.

Primero deberéis cargar los datos proporcionados, que encontraréis en el fichero mt/nld.txt 

In [2]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Definir una función para leer fichero
def read_text(filename):
        file = open(filename, mode='rt', encoding='utf-8')
        text = file.read()
        file.close()
        return text

# Definir una función para procesar los datos
def to_lines(text):
    # Elimina espacios en blanco al principio y al final del texto y luego lo divide en líneas
    sents = text.strip().split('\n')
    
    # Divide cada línea en tabulaciones y toma solo los dos primeros elementos
    sents = [i.split('\t')[:2] for i in sents]
    return sents

# Lee el texto del archivo especificado
data = read_text("/kaggle/input/data-pra2/mt/nld.txt")

# Convierte el texto en una lista de líneas
nld = to_lines(data)
nld = array(nld)

print(nld)

[['Go.' 'Lopen!']
 ['Go.' 'Vooruit.']
 ['Hi.' 'Hoi.']
 ...
 ["If you translate from your second language into your own native language, rather than the other way around, you're less likely to make mistakes."
  'Als je vanuit je tweede taal naar je eigen moedertaal vertaalt, in plaats van andersom, maak je minder snel fouten.']
 ['The Tatoeba Project, which can be found online at tatoeba.org, is working on creating a large database of example sentences translated into many languages.'
  'Het Tatoeba-project, dat je online kan vinden op tatoeba.org, werkt aan een grote gegevensbank met in vele talen vertaalde voorbeeldzinnen.']
 ["If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker."
  'Als iemand die je achtergrond niet kent zegt dat je klinkt als een moedertaalspreker betekent da

Preprocesar los datos, para eliminar puntuaciones y poner en minúscula

In [3]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Elimina puntuaciones de cada elemento 
nld[:, 0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in nld[:, 0]]
nld[:, 1] = [s.translate(str.maketrans('', '', string.punctuation)) for s in nld[:, 1]]

# Convierte a minúsculas cada elemento
for i in range(len(nld)):
    nld[i,0] = nld[i,0].lower()
    nld[i,1] = nld[i,1].lower()

Visualizar los datos resultantes, para tener una idea de como van a ser los datos con los que vamos a trabajar, en concreto ver el tamaño del corpus tanto los vectores del inglés como las del holandes.

In [4]:
#############################################
# SOLUCIÓN                                  #
#############################################

#Visualizar los datos
print(nld)

[['go' 'lopen']
 ['go' 'vooruit']
 ['hi' 'hoi']
 ...
 ['if you translate from your second language into your own native language rather than the other way around youre less likely to make mistakes'
  'als je vanuit je tweede taal naar je eigen moedertaal vertaalt in plaats van andersom maak je minder snel fouten']
 ['the tatoeba project which can be found online at tatoebaorg is working on creating a large database of example sentences translated into many languages'
  'het tatoebaproject dat je online kan vinden op tatoebaorg werkt aan een grote gegevensbank met in vele talen vertaalde voorbeeldzinnen']
 ['if someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker in other words you dont really sound like a native speaker'
  'als iemand die je achtergrond niet kent zegt dat je klinkt als een moedertaalspreker betekent dat dat diegene waarschijnlijk 

Calcular el vocabulario tanto en holandés como en inglés, e imprimir su tamaño. 

In [5]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Definir una función para tokenizar las oraciones
def tokenization(sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer

# Tokenizar las oraciones en inglés y obtener el tamaño del vocabulario en inglés
eng_tokenizer = tokenization(nld[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
print('Tamaño del vocabulario en inglés: %s' % str(eng_vocab_size))

# Tokenizar las oraciones en holandés y obtener el tamaño del vocabulario en holandés
dut_tokenizer = tokenization(nld[:, 1])
dut_vocab_size = len(dut_tokenizer.word_index) + 1
print('Tamaño del vocabulario en holandés: %s' % str(dut_vocab_size))

# Definir la longitud máxima de las secuencias en inglés y holandés
eng_length = 8
dut_length = 8

Tamaño del vocabulario en inglés: 10101
Tamaño del vocabulario en holandés: 14586


Separamos los conjuntos de entrenamiento por idioma y los codificamos. 

In [6]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Definir una función para codificar las secuencias
def encode_sequences(tokenizer, length, lines):
    # Codificar las secuencias con los índices de las palabras
    seq = tokenizer.texts_to_sequences(lines)
    # Hacer el padding
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

# Dividir los datos en conjuntos de entrenamiento y prueba
train, test = train_test_split(nld, test_size=0.001, random_state=12) # División en conjuntos de entrenamiento y prueba

# Codificar las secuencias en el conjunto de entrenamiento
trainX = encode_sequences(dut_tokenizer, dut_length, train[:, 1])  # Codificar las secuencias en holandés
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])  # Codificar las secuencias en inglés

# Codificar las secuencias en el conjunto de prueba
testX = encode_sequences(dut_tokenizer, dut_length, test[:, 1])  # Codificar las secuencias en holandés
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])  # Codificar las secuencias en inglés

Definimos el modelo encoder-decoder basandonos en el notebook visto en la asignatura, e instanciamos el modelo con una capa de embedding para las frases de la lengua origen (inglés) y la dimensión de la última capa como el vocabulario de la lengua destino (holandés)

In [7]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Ajustamos los parametros
max_text_length = 8
embedding_vec_length = 300

# Definimos el modelo
model = Sequential()

# Capa de embedding para las frases en inglés
model.add(Embedding(eng_vocab_size, embedding_vec_length, input_length=max_text_length, mask_zero=True))

# Capa LSTM como encoder
model.add(LSTM(embedding_vec_length))

# Capa de repetición del vector de salida del encoder
model.add(RepeatVector(eng_length))

# Capa LSTM como decoder
model.add(LSTM(embedding_vec_length, return_sequences=True))

# Capa densa de salida con activación softmax
model.add(Dense(len(eng_tokenizer.word_index) + 1, activation='softmax'))

Compilamos el modelo

In [8]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Definir el optimizador RMSprop con una tasa de aprendizaje de 0.001
rms = optimizers.RMSprop(learning_rate=0.001)

# Compilar el modelo con el optimizador RMSprop y la función de pérdida 'sparse_categorical_crossentropy'
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

Entrenamos y guardamos el modelo. 
El modelo puede tardar horas si se hace en CPU, mucho menos si se puede realizar en GPU. Colab permite el uso de GPU en general, si no se hace un uso extensivo, y se va deshabilitando la opción y habilitando segun necesidades. Si se tiene activada siempre penaliza y la desactiva. Para probar si funciona, recomendamos probar de lanzar el entrenamiento solo con una época y ver que funciona, y una vez tenemos claro que el flujo esta funcionando, ya lanzar con muchas más. 

Hemos visto que en Colab, a pesar de que pedimos que el tamaño de sentencia máxima sea 12, no puede cargar el modelo en memoria y recomendamos bajarlo a 4 y el número de "units" a 200, de esta manera si que es capaz de trabajar.

In [9]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Crear un punto de control para guardar los mejores pesos durante el entrenamiento
checkpoint = ModelCheckpoint('dut_en', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Entrenar el modelo y guardar los mejores pesos durante el entrenamiento
history = model.fit(trainX, 
                    trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=50, 
                    batch_size=64, 
                    validation_split=0.2, 
                    callbacks=[checkpoint],
                    verbose=1)

Epoch 1/50
917/919 [============================>.] - ETA: 0s - loss: 4.2259
Epoch 1: val_loss improved from inf to 3.85890, saving model to dut_en
919/919 [==============================] - 38s 31ms/step - loss: 4.2250 - val_loss: 3.8589
Epoch 2/50
918/919 [============================>.] - ETA: 0s - loss: 3.5461
Epoch 2: val_loss improved from 3.85890 to 3.39739, saving model to dut_en
919/919 [==============================] - 27s 29ms/step - loss: 3.5459 - val_loss: 3.3974
Epoch 3/50
916/919 [============================>.] - ETA: 0s - loss: 3.1599
Epoch 3: val_loss improved from 3.39739 to 3.07479, saving model to dut_en
919/919 [==============================] - 26s 28ms/step - loss: 3.1596 - val_loss: 3.0748
Epoch 4/50
919/919 [==============================] - ETA: 0s - loss: 2.8637
Epoch 4: val_loss improved from 3.07479 to 2.85852, saving model to dut_en
919/919 [==============================] - 26s 29ms/step - loss: 2.8637 - val_loss: 2.8585
Epoch 5/50
917/919 [============

Una vez entrenado el modelo, se aplica con el conjunto de test para obtener unas prediciones. 

In [10]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Función para buscar la palabra correspondiente a un índice en el tokenizer
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

# Obtener las predicciones del modelo
preds = np.argmax(model.predict(testX.reshape((testX.shape[0], testX.shape[1]))), axis=-1)

# Resultado de las predicciones
preds_text = []

# Buscamos las predicciones
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], eng_tokenizer)
        if j > 0:
            if (t == get_word(i[j-1], eng_tokenizer)) or (t == None):
                temp.append('')
            else:
                temp.append(t)
        else:
            if(t == None):
                temp.append('')
            else:
                temp.append(t) 

    preds_text.append(' '.join(temp))

3/3 [==============================] - 2s 8ms/step


Visualizamos los resultados de las predicciones con los valores esperados. Los resultados son curiosos, no podríamos usar este modelo para un entorno real como vais a poder ver. 

Pregunta: ¿Porque creéis que no son buenos, y como creéis que podrían obtenerse mejores resultados?

In [14]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Crear un DataFrame con las columnas 'actual' y 'predicted'
results = pd.DataFrame({'actual': test[:, 0], 'predicted': preds_text})
results

,actual,predicted
0,i feel lousy,im feel love
1,do you want red or white wine with your meal,if your black and name or the white
2,you look satisfied,you look to
3,is that all right with you,are you ok good
4,you cant live on that island,you cant live in that glass
...,...,...
69,was i wrong,was i wrong
70,its been fun,its been fun
71,did that happen recently,did the happened
72,you were late werent you,you youre guys late


Los resultados que se muestran indican que el modelo no está generando predicciones precisas y coherentes con los valores esperados. Hay varias posibles razones por las que esto podría estar sucediendo y cómo se podrían obtener mejores resultados:

1. **Tamaño de los datos de entrenamiento:** El tamaño del conjunto de datos de entrenamiento puede no ser suficiente para capturar la complejidad y variabilidad del problema de traducción automática. Obtener un conjunto de datos más grande y diverso podría ayudar a mejorar el rendimiento del modelo.

2. **Longitud máxima de secuencia:** En el código, se establece una longitud máxima de secuencia de 8 palabras tanto para el inglés como para el holandés. Esta longitud podría ser demasiado limitada para capturar el contexto completo de las oraciones y dificultar la generación de traducciones precisas. Aumentar la longitud máxima de secuencia podría permitir que el modelo capture mejor las relaciones y estructuras más largas en los textos.

3. **Hiperparámetros del modelo:** Los hiperparámetros del modelo, como la dimensión de embedding, el número de capas LSTM y el tamaño del lote, pueden tener un impacto significativo en el rendimiento del modelo. Ajustar y optimizar adecuadamente estos hiperparámetros podría mejorar la capacidad del modelo para capturar y aprender las características del idioma.

4. **Entrenamiento y tiempo de entrenamiento:** El número de épocas de entrenamiento y el tamaño del lote utilizado en el código proporcionado podrían no ser óptimos para este problema específico. Ajustar estos parámetros y aumentar el número de épocas podría permitir un entrenamiento más exhaustivo del modelo y una mejora en las predicciones.

5. **Arquitectura del modelo:** El modelo utilizado en el código es una arquitectura básica de encoder-decoder con capas LSTM. Esta arquitectura puede no ser lo suficientemente compleja o adecuada para el problema de traducción automática en cuestión. Se podrían explorar arquitecturas más avanzadas, como el uso de mecanismos de atención, para capturar mejor las relaciones entre las palabras y mejorar la calidad de las traducciones.

## 1.2 TA con Embeddings preentrenados (2 puntos)

En este apartado repetiremos el ejercicio anterior cargando a la capa de embedding los pesos d'un modelo GloVe entrenado para el inglés. 

Empezamos cargando el modelo GloVe para el inglés. Podéis usar 'glove.42B.300d.txt'. Se puede bajar de aquí: https://nlp.stanford.edu/projects/glove/

In [9]:
embeddings_index = {}
f = open('/kaggle/input/dataTA/glove.42B.300d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print(len(embeddings_index))

1917494


A continuación, tenemos que construir la matriz de embeddings. 
Para no cargar todo el vocabulario del modelo, podemos filtrar solo aquellas entradas presentes en el vocabulario del tokenizador que usaremos. Además, tenemos de incluir en la matriz de vectores correspondientes los índices de las entradas (palabras) que no encontremos en el modelo glove cargado. Estos vectores se suelen inicializar con 0s o con el resultado de una distribución N(0,1)

Por ejemplo, si nuestro tokenizador se llamara `eng_tokenizer` podríamos hacer:

In [10]:
embedding_matrix = np.zeros((len(eng_tokenizer.word_index) + 1, 300))
for word, i in eng_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Para inicializar una capa de embeddings con pesos predefinidos se utiliza el argumento `weights`. Además, como no queremos que se modifiquen los pesos, marcamos el argumento `trainable` como `False`. 

Siguiendo con nuestro ejemplo, haríamos:

In [11]:
from keras.layers import Embedding

embedding_layer = Embedding(len(eng_tokenizer.word_index) + 1,
                            embedding_vec_length,
                            weights=[embedding_matrix],
                            input_length=max_text_length,
                            trainable=False,
                            mask_zero=True)

Implementa y entrena de nuevo un modelo de traducción automática del inglés al holandés de forma similar, esta vez cargando los pesos de la capa embedding a partir del modelo Glove preentrenado en inglés y disponible en 'glove.42B.300d.txt'.


In [12]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Se establecen los pesos de la capa de embeddings del modelo utilizando la matriz de embeddings predefinida
model.layers[0].set_weights([embedding_matrix])

# Se marca la capa de embeddings como no entrenable, lo que significa que los pesos no se actualizarán durante el entrenamiento
model.layers[0].trainable = False

# Se muestra un resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 300)            3030300   
                                                                 
 lstm (LSTM)                 (None, 300)               721200    
                                                                 
 repeat_vector (RepeatVector  (None, 8, 300)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 8, 300)            721200    
                                                                 
 dense (Dense)               (None, 8, 10101)          3040401   
                                                                 
Total params: 7,513,101
Trainable params: 4,482,801
Non-trainable params: 3,030,300
______________________________________

Entrenamos y guardamos el modelo. Otra vez, aunque este entrenamiento es quizá un "poco" más liviano que el anterior, recomendamos el uso de GPU si es viable.

In [13]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Se define el optimizador RMSprop con una tasa de aprendizaje de 0.001
rms = optimizers.RMSprop(learning_rate=0.001)

# Se compila el modelo utilizando el optimizador RMSprop y la función de pérdida 'sparse_categorical_crossentropy'
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

# Se crea un punto de control para guardar el mejor modelo según la pérdida de validación
checkpoint = ModelCheckpoint('dut_en', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Se entrena el modelo
history2 = model.fit(trainX, 
                     trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                     epochs=50, 
                     batch_size=64, 
                     validation_split=0.2, 
                     callbacks=[checkpoint],
                     verbose=1)


Epoch 1/50
918/919 [============================>.] - ETA: 0s - loss: 4.1412
Epoch 1: val_loss improved from inf to 3.72254, saving model to dut_en
919/919 [==============================] - 39s 32ms/step - loss: 4.1409 - val_loss: 3.7225
Epoch 2/50
916/919 [============================>.] - ETA: 0s - loss: 3.4276
Epoch 2: val_loss improved from 3.72254 to 3.28532, saving model to dut_en
919/919 [==============================] - 29s 32ms/step - loss: 3.4271 - val_loss: 3.2853
Epoch 3/50
916/919 [============================>.] - ETA: 0s - loss: 3.0593
Epoch 3: val_loss improved from 3.28532 to 3.02064, saving model to dut_en
919/919 [==============================] - 29s 31ms/step - loss: 3.0590 - val_loss: 3.0206
Epoch 4/50
918/919 [============================>.] - ETA: 0s - loss: 2.8001
Epoch 4: val_loss improved from 3.02064 to 2.88180, saving model to dut_en
919/919 [==============================] - 31s 34ms/step - loss: 2.7998 - val_loss: 2.8818
Epoch 5/50
916/919 [============

Aplicar el modelo y visualizar los resultados a partir de las prediciones obtenidas con este nuevo modelo.

In [14]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Obtener las predicciones del modelo
preds = np.argmax(model.predict(testX.reshape((testX.shape[0],testX.shape[1]))), axis=-1)

# Resultados
preds_text = []

# Buscamos las predicciones
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], eng_tokenizer)
        if j > 0:
            if (t == get_word(i[j-1], eng_tokenizer)) or (t == None):
                     temp.append('')
            else:
                     temp.append(t)
        else:
            if(t == None):
                temp.append('')
            else:
                temp.append(t) 

    preds_text.append(' '.join(temp))

# Crear un DataFrame con las columnas 'actual' y 'predicted'
print(pd.DataFrame({'actual' : test[:,0], 'predicted' : preds_text}))

3/3 [==============================] - 2s 7ms/step
                                          actual                 predicted
0                                   i feel lousy              i feel      
1   do you want red or white wine with your meal  you  or white   of white
2                             you look satisfied       you look right     
3                     is that all right with you       do you think at    
4                   you cant live on that island  you cant live on in on  
..                                           ...                       ...
69                                   was i wrong               i  got     
70                                  its been fun            its been      
71                      did that happen recently   did it happened the    
72                      you were late werent you    you guys didnt get    
73                        ive hired an assistant       i need an sorry    

[74 rows x 2 columns]


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>(Opcional) Análisis:</strong> Explica cuales són las principales diferencias entre los dos modelos entrenados. ¿Como podríamos mejorar los resultados de esta tarea en concreto?
</div>

Las principales diferencias entre los dos modelos entrenados son las siguientes:

1. **Modelo sin preentrenamiento de embeddings:** En este modelo, se utilizó una capa de embeddings que se inicializó aleatoriamente y se entrenó junto con el resto del modelo durante el proceso de entrenamiento. Los pesos de los embeddings se ajustaron para adaptarse a los datos específicos de la tarea de traducción.

2. **Modelo con preentrenamiento de embeddings:** En este modelo, se cargaron los pesos de la capa de embeddings a partir de un modelo GloVe preentrenado en inglés. Esto permitió utilizar vectores de palabras preentrenados que capturan información semántica y sintáctica general del idioma inglés. Los pesos de la capa de embeddings se mantuvieron fijos durante el entrenamiento del modelo de traducción.

Para mejorar los resultados de esta tarea en concreto, se pueden considerar las siguientes estrategias:

1. **Aumentar el tamaño del conjunto de entrenamiento:** Obtener más datos de entrenamiento puede ayudar al modelo a capturar patrones y regularidades lingüísticas de manera más efectiva.

2. **Aumentar la complejidad del modelo:** Se puede explorar la posibilidad de utilizar arquitecturas de modelos más complejas, como modelos con más capas o unidades de LSTM más grandes, para capturar relaciones más complejas en los datos de entrada.

3. **Ajustar los hiperparámetros:** Se pueden experimentar con diferentes valores de hiperparámetros, como la tasa de aprendizaje, el tamaño del lote y el número de épocas de entrenamiento, para encontrar la configuración óptima que maximice el rendimiento del modelo.

4. **Utilizar técnicas de regularización:** Se pueden aplicar técnicas de regularización, como la regularización L1 o L2, la eliminación de dropout o la normalización por lotes, para reducir el sobreajuste y mejorar la generalización del modelo.

5. **Explorar modelos de traducción más avanzados:** En lugar de utilizar un modelo encoder-decoder básico, se pueden explorar modelos más avanzados como Transformer, que han demostrado excelentes resultados en tareas de traducción automática.

# PARTE 2


# 2. Classificación de notícias (4 puntos)

---



En este apartado planteamos el uso de las arquitecturas vistas hasta el momento para crear un clasificador de notícias en inglés. 
En concreto usaremos este dataset:
https://www.kaggle.com/datasets/rmisra/news-category-dataset
Es un dataset de clasificación que tiene 42 categorías. Vamos a enfocarnos en 6 de ellas. En concreto las siguientes:
+ HEALTHY LIVING
+ QUEER VOICES
+ FOOD & DRINK
+ BUSINESS
+ COMEDY
+ SPORTS


## 2.1 Preparar datos clasificación notícias (1 punto)

Lo primero que haremos será obtener el dataset, coger el archivo , leerlo desde un dataframe pandas, coger las columnas "headline" y "category".

Samplear el corpus y coger sólo las **1000** primeras entradas para que el script vaya más rápido. Más adelante podremos realizar pruebas para ver si añadiendo más entradas tenemos mejores resultados.Obviamente las entradas, luego se repartiran por las categorías que hemos escogido. 

In [27]:
#############################################
# SOLUCIÓN                                 #
#############################################

# Lee el archivo JSON de noticias y selecciona las columnas "headline" y "category"
json = pd.read_json('/kaggle/input/data-pra2/class/News_Category_Dataset_v3.json', lines=True)[['headline','category']]

# Filtra el dataframe para mantener solo las filas que pertenecen a las categorías de interés
json = json[json['category'].isin(['HEALTHY LIVING','QUEER VOICES','FOOD & DRINK','BUSINESS','COMEDY','SPORTS'])]

# Realiza un muestreo aleatorio de 1000 filas del dataframe
json = json.sample(1000)

# Imprime las primeras filas del dataframe resultante
json.head()


,headline,category
27275,White Supremacists Are Using Genetic Ancestry ...,HEALTHY LIVING
190610,The 5 Most Bafflingly Racist Shows On TV Right...,COMEDY
192879,Do You Know Your Classic Ice Cream Truck Treat...,FOOD & DRINK
46770,It's The Horrifying Tale Of The Drunk Girl Who...,COMEDY
20380,"7+ Reasons Why Bisexual, Pansexual, Fluid, And...",QUEER VOICES


Visualizad la distribución de textos por clase.


In [28]:
#############################################
# SOLUCIÓN                                  #
#############################################

#Visualizamos la distribución por clase
category_counts = json['category'].value_counts()
print(category_counts)

HEALTHY LIVING    185
QUEER VOICES      183
COMEDY            172
FOOD & DRINK      172
BUSINESS          156
SPORTS            132
Name: category, dtype: int64


Preparad y preprocesad los datos para el entrenamiento. Utilizaremos one-hot encoding por las etiquetas.

In [29]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Preparación de los textos
json['headline'] = json['headline'].apply(lambda x: str(x.translate(str.maketrans('','',string.punctuation)).lower()))

def encod(x):
    # Función para codificar las etiquetas
    if x == 'HEALTHY LIVING':
        return 0
    elif x == 'QUEER VOICES':
        return 1
    elif x == 'BUSINESS':
        return 2
    elif x == 'FOOD & DRINK':
        return 3
    elif x == 'COMEDY':
        return 4
    elif x == 'SPORTS':
        return 5

# Codificación de las etiquetas
json['category'] = json['category'].apply(lambda x: encod(x))


## 2.2 Preparar datos y embeddings para entrenar (1 punto)
La idea del modelo de clasificación que queremos implementar es más simple que la del encoder-decoder usado en el apartado 1.

El modelo debe consistir sólo en:

- una capa embedding con los pesos del modelo Glove preentrenado para el inglés disponible en el archivo 'glove.42B.300d.txt'
- una capa LSTM con un número de units a elegir (por ejemplo, 300)
- una capa Dense con una dimensión de salida que tiene el número de categorías con las que queremos clasificar (en este caso, 6).
- Además, como loss function `loss` utilizaremos 'categorical_crossentropy' y como `optimizer`, 'adam'.

Primeramente creamos un tokenizador para las frases del clasificador

In [30]:
#############################################
# SOLUCIÓN                               #
#############################################

# Creación del tokenizador para las frases del clasificador
eng_tokenizer = tokenization(json['headline'].values)

# Tamaño del vocabulario
eng_vocab_size = len(eng_tokenizer.word_index) + 1

# Longitud máxima de las frases de entrada
eng_length = 8



Cargamos el siguiente modelo GloVe para el inglés. Lo hemos utilizado en la parte 1.

In [33]:
#############################################
# SOLUCIÓN                                #
#############################################

# Vemos el tamaño para ver si el modelo esta cargado
embedding_matrix.shape

(10101, 300)

Una vez cargado el modelo de GloVe definimos la capa de Embedding con todos sus pesos


In [34]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Definimos el modelo
model = Sequential()
# Capa de Embedding
model.add(Embedding(10101, 300, weights=[embedding_matrix], input_length=8, trainable=False))
# Capa LSTM
model.add(LSTM(300))
# Capa Dense
model.add(Dense(6))
# Capa de activación softmax
model.add(Activation('softmax'))
# Compilamos el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Preparamos el corpus de entrenamiento y test, usando el model_selection de sklearn, y el onehot_encoded para las clases. Usamos 80% para train. 

In [31]:
#############################################
# SOLUCIÓN                                  #
#############################################

# División del dataset en conjuntos de entrenamiento y prueba
xtrain, xtest, ytrain, ytest = train_test_split(json['headline'].values, json['category'].values, random_state=42, test_size=0.2, shuffle=True)

Codificar los vectores de entrada para el train y para el text

In [32]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Codificación de las secuencias de entrada
trainX = encode_sequences(eng_tokenizer, eng_length, xtrain)
testX = encode_sequences(eng_tokenizer, eng_length, xtest)

## 2.3 Definir el modelo y entrenar (1 punto).


El modelo debe consistir sólo en:

+ una capa embedding con los pesos del modelo GloVe preentrenado para el inglés disponible en el archivo 'glove.42B.300d.txt'
+ una capa LSTM con un número de units a elegir (por ejemplo, 300)
+ una capa Dense con una dimensión de salida que tiene el número de categorías con las que queremos clasificar (en este caso, 6).
    
Además, como loss function loss utilizaremos 'categorical_crossentropy' y como optimizer, 'adam'.

In [35]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Se muestra un resumen del modelo
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 8, 300)            3030300   
                                                                 
 lstm_3 (LSTM)               (None, 300)               721200    
                                                                 
 dense_2 (Dense)             (None, 6)                 1806      
                                                                 
 activation_1 (Activation)   (None, 6)                 0         
                                                                 
Total params: 3,753,306
Trainable params: 723,006
Non-trainable params: 3,030,300
_________________________________________________________________


Compilar el modelo

In [36]:
#############################################
# SOLUCIÓN                                  #
#############################################

#Compilamos el modelo
rms = optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

Entrenar y guardar el modelo. En esta sección aunque sea recomendable usar GPU, con CPU también se puede obtener el resultado sin tener que esperar "mucho" tiempo

In [39]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Entrenamos el modelo
history2 = model.fit(trainX, ytrain, epochs=100, batch_size=32, validation_split = 0.2, verbose=1)

Epoch 1/100
20/20 [==============================] - 1s 8ms/step - loss: 0.4264 - val_loss: 4.3279
Epoch 2/100
20/20 [==============================] - 0s 7ms/step - loss: 0.0215 - val_loss: 4.6433
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.0021 - val_loss: 5.1058
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 5.6281e-04 - val_loss: 5.7006
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 1.8562e-04 - val_loss: 6.2806
Epoch 6/100
20/20 [==============================] - 0s 9ms/step - loss: 7.2143e-05 - val_loss: 6.7916
Epoch 7/100
20/20 [==============================] - 0s 11ms/step - loss: 2.7207e-05 - val_loss: 7.4083
Epoch 8/100
20/20 [==============================] - 0s 9ms/step - loss: 0.3312 - val_loss: 6.8895
Epoch 9/100
20/20 [==============================] - 0s 11ms/step - loss: 0.2120 - val_loss: 4.7998
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0036 - val_loss

## 2.4 Evaluar el modelo (1 punto)

Se evalua el modelo y se obtienen sus diferentes métricas. 

In [40]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Evaluamos el modelo
print(classification_report(ytest, [np.argmax(i) for i in model.predict(testX)], target_names=['HEALTHY LIVING', 'QUEER VOICES', 'BUSINESS', 'FOOD & DRINK', 'COMEDY', 'SPORTS']))

7/7 [==============================] - 0s 2ms/step
                precision    recall  f1-score   support

HEALTHY LIVING       0.25      0.24      0.25        37
  QUEER VOICES       0.31      0.33      0.32        30
      BUSINESS       0.17      0.09      0.11        35
  FOOD & DRINK       0.34      0.35      0.35        37
        COMEDY       0.26      0.31      0.28        36
        SPORTS       0.21      0.28      0.24        25

      accuracy                           0.27       200
     macro avg       0.26      0.27      0.26       200
  weighted avg       0.26      0.27      0.26       200



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> ¿Que pasaría si el modelo de clasificación lo entrenaramos con más datos? Estamos escogiendo solo los 1000 primeros, son muy pocos ejemplos y se entrena super rápido. Que pasaría con 5000? y 10000?
Y si no usaramos los embeddings de GloVe? Que nos aportan ambas cosas?
¿Si usamos más datos hacen falta usar los embeddings?

Expresar vuestra opinión con experimentación acorde para obtener las conclusiones.





Si entrenamos el modelo de clasificación con más datos, como 5000 o 10000 ejemplos en lugar de los 1000 actuales, es probable que mejore el rendimiento y la capacidad de generalización del modelo. Al tener más ejemplos de entrenamiento, el modelo tendrá la oportunidad de aprender patrones más complejos y representativos, lo que puede resultar en una mayor precisión en las predicciones.

El uso de los embeddings de GloVe aporta beneficios significativos al modelo. Los embeddings preentrenados capturan la información semántica y sintáctica de las palabras, lo que permite al modelo comprender mejor el significado de las frases. Al utilizar estos embeddings, el modelo se beneficia de la transferencia de conocimiento obtenida del modelo preentrenado en un corpus mucho más grande.

Sin embargo, es importante destacar que el impacto de los embeddings y la cantidad de datos depende del contexto y la tarea específica. En algunos casos, cuando se dispone de suficientes datos de entrenamiento específicos para la tarea, los embeddings preentrenados pueden no ser tan necesarios y el modelo puede aprender representaciones adecuadas por sí mismo.

Para obtener conclusiones más sólidas, sería recomendable realizar experimentos comparativos utilizando diferentes cantidades de datos de entrenamiento (por ejemplo, 1000, 5000 y 10000 ejemplos) y evaluar el rendimiento del modelo con y sin el uso de los embeddings de GloVe. Esto permitiría determinar cómo afecta la cantidad de datos y la presencia de embeddings en el rendimiento del modelo en la tarea específica de clasificación de noticias.

# PARTE 3



# 3. Detección de NER and NEL (2 puntos)

En esta parte detectarermos entidades numeradas utilizando tanto SpaCy con transformers como transformers simplemente. En el segundo caso usando una librería llamada simple transformers.

Por otro lado, también haremos Named Entity Linking (NEL) donde buscaremos entidades linkadas a una base de conocimiento (KB), en este caso DBpedia. Encontraremos los enlaces a Wikpedia de ciertas entidades del texto, utilizando DBPedia Spotlight.

## 3.1 Detección de NER en Spacy (1 punto)

Detección de entidades nombradas (NER) usando spaCy. En esta sección usaremos spaCy para detectar NER. A partir de un corpus de CONLL 2003, lo reeentrenaremos y de esta manera afinaremos su cobertura para estas clases. 

Instalamos spacy y mos modelos de lenguaje que necesitemos. 

In [3]:
!pip install spacy==3.2.0
!pip install -U spacy-transformers==1.1.7
!pip install -U thinc==8.0.12
!python -m spacy download en_core_web_sm-3.2.0 --direct

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 44.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 46.6 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.9
    Uninstalling thinc-8.1.9:
      Successfully uninstalled thinc-8.1.9
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled sp

Definimos un par de funciones que nos va a permitir imprimir los resultados de la detección de NER de forma muy interpretable

In [4]:
def get_tokens_to_print(model, text):
  """Print tokens of the text and its relevant attributes.

    Parameters:
      model (spaCy model): spaCy model used for tokenization
      text (str):  text to transform in a spaCy doc class.

    Returns: ---
  """
  doc = model(text)
  print (f"The text:\n\n{get_text_to_print(text)}\n\nwas converted in a spaCy object: {type(doc)}\n")
  print (f"Token-based analysis. Each token is a spaCy object: {type(doc[0])}\n")
  
  # We obtain rows to print: headers and content
  rows  = []
  # head_align: List of tuples. Each tuple: heather and its alignment when printing
  head_align  = [('Token', '<'), ('Lemma', '<'), ('Syntactic parent', '<'), ('#Tok', '>'), ('Chr_Start', '>'), ('Chr_End', '>'), ('POS', '<'), 
                 ('TAG', '<'), ('TAG meaning:', '<'), ('ENT', '<'), ('DEP', '<'), ('DEP meaning:', '<')]   
  head, align = list(zip(*head_align))  
  rows.append(head)                           # Header
  rows.append(['='*len(i) for i in head])     # Underline headers
  for tok in doc:
    rows.append([tok.text, tok.lemma_, tok.head.text, str(tok.i), str(tok.idx), str(tok.idx+len(tok)-1), tok.pos_, 
                 tok.tag_, str(spacy.explain(tok.tag_))[:20], tok.ent_type_, tok.dep_, str(spacy.explain(tok.dep_))[:20]])
  
  # Width of each column: the witdh of the longest element
  columns       = zip(*rows)     
  column_widths = [max(len(i) for i in col) for col in columns]

  # Print the files with alignment
  for row in rows:
    print(*[f"{row[i]:{align[i]}{column_widths[i]}}  " for i in range(0, len(row))])

In [5]:
def get_text_to_print(text):
  """Format given text.

    Parameters:
      text (str): text to print

    Returns:
      str: text formatted in 100 character lines with an initial line numbering the characters
  """
  line_length = 100
  line_poss   = "     1-------10--------20--------30--------40--------50--------60--------70--------80--------90-------100"
  text        = text.replace("\n", " ")     # In order to avoid that the \n character produces a line change.
  text        = text.replace("\r", " ")     # In wikipedia texts we have detected the character '\r' that, if interpreted, may induce some printing problems.
  text_format = "\n".join([ f"{i//line_length:<5}{text[i:i+line_length]}"  for i in range(0, len(text), line_length) ])
  return line_poss + "\n" + text_format + "\n" + line_poss

Cargamos el modelo "en_core_web_sm"

In [7]:
#############################################
# SOLUCIÓN                                  #
#############################################

import spacy

# Cargamos el modelo
nlp = spacy.load("en_core_web_sm")

Convertimos un texto en objecto 'Doc' de spaCy y visualizamos los resultados de analizar este texto a nivel de POS, NER, ENT, DEP... 

In [8]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Función para analizar un texto
def analyze_text(text):
    doc = nlp(text)
    for token in doc:
        print(f"Token: {token.text}\tLemma: {token.lemma_}\tPOS: {token.pos_}\tNER: {token.ent_type_}\tDependency: {token.dep_}")

# Uso de la función anterior
text = "Apple Inc. is looking to buy a startup in the autonomous vehicle industry."
analyze_text(text)

Token: Apple	Lemma: Apple	POS: PROPN	NER: ORG	Dependency: compound
Token: Inc.	Lemma: Inc.	POS: PROPN	NER: ORG	Dependency: nsubj
Token: is	Lemma: be	POS: AUX	NER: 	Dependency: aux
Token: looking	Lemma: look	POS: VERB	NER: 	Dependency: ROOT
Token: to	Lemma: to	POS: PART	NER: 	Dependency: aux
Token: buy	Lemma: buy	POS: VERB	NER: 	Dependency: xcomp
Token: a	Lemma: a	POS: DET	NER: 	Dependency: det
Token: startup	Lemma: startup	POS: NOUN	NER: 	Dependency: dobj
Token: in	Lemma: in	POS: ADP	NER: 	Dependency: prep
Token: the	Lemma: the	POS: DET	NER: 	Dependency: det
Token: autonomous	Lemma: autonomous	POS: ADJ	NER: 	Dependency: amod
Token: vehicle	Lemma: vehicle	POS: NOUN	NER: 	Dependency: compound
Token: industry	Lemma: industry	POS: NOUN	NER: 	Dependency: pobj
Token: .	Lemma: .	POS: PUNCT	NER: 	Dependency: punct


Entrenar un nuevo modelo de NER con los datos de CONLL2003

Convertimos los ficheros conll03 (train y valid) a formato spaCy. 
El corpus lo hemos obtenido de aquí:
https://github.com/Hironsan/anago

Tip: spacy contiene funciones que permiten convertir de formato conll al formato compilado que necesitan el módulo de train de spaCy. 


In [63]:
#############################################
# SOLUCIÓN                                  #
#############################################

from spacy.tokens import DocBin

# Ruta a los archivos CONLL
train_file = "/kaggle/input/data-pra2/ner/conll03/train.txt"
valid_file = "/kaggle/input/data-pra2/ner/conll03/valid.txt"

# Rutas de salida para los archivos en formato spaCy
train_output_dir = "/kaggle/working/"
valid_output_dir = "/kaggle/working/"

# Convertir archivo de entrenamiento a formato spaCy
!python -m spacy convert {train_file} {train_output_dir} --converter conll --file-type "spacy"

# Convertir archivo de validación a formato spaCy
!python -m spacy convert {valid_file} {valid_output_dir} --converter conll --file-type "spacy"

# Cargar los datos de entrenamiento y validación en variables
train_data = DocBin().from_disk("/kaggle/working/train.spacy")
valid_data = DocBin().from_disk("/kaggle/working/valid.spacy")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (14041 documents):
/kaggle/working/train.spacy
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3250 documents):
/kaggle/working/valid.spacy


Descargar el modelo 'en_core_web_trf'

In [10]:
!python -m spacy download en_core_web_trf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.2.0/en_core_web_trf-3.2.0-py3-none-any.whl#egg=en_core_web_trf==3.2.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 MB 2.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [11]:
!python -m spacy validate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.2.0) =================
ℹ spaCy installation: /opt/conda/lib/python3.10/site-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_trf   >=3.2.0,<3.3.0   3.2.0   ✔
en_core_web_sm    >=3.2.0,<3.3.0   3.2.0   ✔
en_core_web_lg    >=3.5.0,<3.6.0   3.5.0   --> 3.2.0     


============================== Install updates ==============================
Use the following commands to update the packages:
python -m spacy download en_core_web_lg



In [12]:
print (f"Spacy version installed: {spacy.__version__}")

Spacy version installed: 3.2.0


Entrenar usando la función train de spaCy a partir del modelo 'en_core_web_trf'. 
Usar el fichero de configuración adjunto y modificar las cosas que consideréis oportunas. La versión entregada ya funciona pero se puede customizar si hay interés. 

In [83]:
#############################################
# SOLUCIÓN                                  #
#############################################

from spacy.cli.train import train

#Entrenar el modelo con la configuración nueva
train("/kaggle/input/configuracionmodificada/config_Modificado.cfg", overrides={"paths.train": "/kaggle/working/train.spacy", "paths.dev": "/kaggle/working/valid.spacy"}, use_gpu= 0, output_path ="/kaggle/working/")


ℹ Saving to output directory: /kaggle/working
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================


[2023-06-03 21:07:49,582] [INFO] Set up nlp object from config
[2023-06-03 21:07:49,619] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-03 21:07:49,621] [INFO] Resuming training for: ['ner', 'transformer']
[2023-06-03 21:07:49,630] [INFO] Created vocabulary
[2023-06-03 21:07:52,010] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_trf) has no vectors. This is almost certainly a mistake.
[2023-06-03 21:07:52,015] [INFO] Added vectors: en_core_web_trf
[2023-06-03 21:07:52,056] [INFO] Finished initializing nlp object
[2023-06-03 21:07:52,057] [INFO] Initialized pipeline components: []


✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0         480.74    345.40    7.88    6.40   10.27    0.08
  1     200       38910.36  25996.48   60.22   67.55   54.33    0.60
  2     400        4289.68  12146.00   70.58   70.69   70.46    0.71
  3     600        2295.85   5294.50   93.28   92.85   93.72    0.93
  4     800        1526.68   1930.17   94.69   94.62   94.75    0.95
  5    1000         902.21   1114.27   95.62   95.33   95.91    0.96
  6    1200         843.79    952.09   95.68   95.54   95.83    0.96
  7    1400         685.02    797.04   94.80   94.59   95.02    0.95
✔ Saved pipeline to output directory
/kaggle/working/model-last


Predecir una frase de ejemplo con el nuevo modelo y visualizar los resultados. 
Ojo que colab, no le gusta cargar modelos desde paths, solo si estan en local, con lo que recomendamos, generar el modelo en drive, guardarlo y luego subir la mejor versión para cargarlo desde de aquí. 


In [86]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Obtener el modelo
nlp_predict = spacy.load("/kaggle/working/model-best")

# Frase a predecir
text = "John works at Apple Inc. in Cupertino."

# Realizar al predicción
doc = nlp_predict(text)

# Mostrar el resultado
spacy.displacy.render(doc, style="ent", jupyter=True) 

Evaluar los resultados obtenidos y calcular las métricas

In [87]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Ruta a los archivos CONLL
test_file = "/kaggle/input/data-pra2/ner/conll03/test.txt"

# Rutas de salida para los archivos en formato spaCy
test_output_dir = "/kaggle/working/"

# Convertir archivo de entrenamiento a formato spaCy
!python -m spacy convert {test_file} {test_output_dir} --converter conll --file-type "spacy"

test_data = DocBin().from_disk("/kaggle/working/test.spacy")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3453 documents):
/kaggle/working/test.spacy


In [99]:
####### CÓDIGO OBTENIDO EN PARTE DE LOS EJEMPLOS DEL TEMARIO ###########
def score_ner(examples):
    """Evaluate spaCy model with the spaCy Scorer class.

    Parameters:
      examples (list): list of 'example' objects spaCy: documents spaCy annotated (predicted and gold reference).

    Returns: 
      dict: result of the evaluation. See dictionary format at: https://spacy.io/api/scorer#score_spans   
    """
    scorer = Scorer()
    # Evaluate 'examples' and return the evaluation result
    return scorer.score_spans(examples, "ents")


def print_metrics(metrics, model):
    """Print results of the spaCy entity evaluation.

    Parameters:
      metrics (dict): results of the spaCy entity evaluation; dictionary with spaCy format (see 'returns' in https://spacy.io/api/scorer#score_spans)
      model (spaCy model): spaCy model for NER

    Returns: --- 
    """
    df = pd.DataFrame([metrics["ents_p"], metrics["ents_r"], metrics["ents_f"]], columns=["Total"], index=["Precision", "Recall", "F-score"])
    for entity, scores in metrics["ents_per_type"].items():
        df[entity] = [scores["p"], scores["r"], scores["f"]]
    
    df = (df * 100).round(decimals=2)

    print(f"\nEvaluation of the model: '{model.meta['lang']+'_'+model.meta['name']}'")
    print("===============================================\n")
    print(df)
    print("\n")


def ner_evaluation(model, examples):
    """Evaluate NER with spaCy. Print the evaluation.

    Parameters:
      model (spaCy model): spaCy model for NER
      examples (list): list of spaCy 'example' objects (each 'example' contains text and predicted/gold reference entities)

    Returns: 
      None
    """
    if model is None or examples is None:
        print("ner_evaluation must have valid arguments")
        return
  
    metrics = score_ner(examples)
    print_metrics(metrics, model)

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Cargar el modelo de spaCy
nlp_predict = spacy.load("/kaggle/working/model-best")

# Ruta al archivo de prueba en formato .spacy
test_data_path = "/kaggle/working/test.spacy"

# Cargar los ejemplos de prueba desde el archivo .spacy
test_examples = list(spacy.load(test_data_path).get_pipe("ner").pipe)

# Realizar la evaluación del NER
ner_evaluation(model=nlp_predict, examples=test_examples)

## 3.3 NEL (1 punto)

En esta sección, la idea es obtener los enlaces en la DBpedia spotlight relacionados con las entidades que se han obtenido de NER usando spaCy. 



Desarrolla una función que dado un texto, obtenga automáticamente las entidades relacionadas en el DBpedia Spotlight.
URL de acceso a la API, DBPedia inglés: https://www.dbpedia-spotlight.org/api o https://www.dbpedia-spotlight.org/api/en

In [9]:
#############################################
# SOLUCIÓN                                  #
#############################################

import requests
import urllib.parse

# Función para obtener DPEDIA-LINKS
def get_dbpedia_links(text):
    
    # URL
    base_url = 'https://api.dbpedia-spotlight.org/en/annotate'
    
    # Parametros
    query_params = {
        'text': text,
        'confidence': 0.5
    }
    
    # Codificar el texto para incluirlo en la URL
    encoded_text = urllib.parse.quote(text)
    
    # Codificar los parámetros de la consulta
    encoded_params = urllib.parse.urlencode(query_params)
    
    # Construir la URL completa con el texto codificado y los parámetros
    url = f"{base_url}?text={encoded_text}&{encoded_params}" 
    headers = {'accept': 'application/json'}
    
    try:
        # Realizar la solicitud GET a la API de DBpedia Spotlight
        response = requests.get(url, headers=headers)
        
         # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            
            # Obtener las anotaciones (recursos) de la respuesta JSON
            annotations = response.json().get('Resources', [])
            
             # Extraer los enlaces (URIs) de las anotaciones
            links = [ann['@URI'] for ann in annotations] 
            return links
        else:
            
             # Mostrar el código de estado de la respuesta en caso de error
            print(f'Error accessing DBpedia Spotlight API: {response.status_code}')
            
    except requests.exceptions.RequestException as e:
        
        # Mostrar el error de solicitud en caso de excepción
        print(f'Error accessing DBpedia Spotlight API: {e}')  
    return []

# Texo a analizar
text = 'Elizabeth II has been the Queen of England.'
print("Texto:", text)


try:
    # Obtener los enlaces de DBpedia Spotlight para el texto dado
    dbpedia_links = get_dbpedia_links(text)
    
    # Imprimir los enlaces obtenidos
    print('DBpedia links:', dbpedia_links)  
except Exception as e:
    
    # Mostrar el error en caso de excepción
    print(f'Error getting DBpedia links: {e}')  


Texto: Elizabeth II has been the Queen of England.
DBpedia links: ['http://dbpedia.org/resource/Elizabeth_II', 'http://dbpedia.org/resource/England']
